In [22]:
from dotenv import dotenv_values
from requests import get
from requests.models import HTTPError
from bs4 import BeautifulSoup

import pandas as pd

In [23]:
CONFIG = dotenv_values('.env')

In [38]:
def requisicao(url):
    cookies = {'caelum.login.token':CONFIG['COOKIE']}
    try:
        response = get(url, cookies=cookies)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Um erro HTTP ocorreu: {http_err}')
    except Exception as err:
        print(f'Um erro ocorreu: {err}')
    else:
        soup = BeautifulSoup(response.text,'html.parser')
        return soup

In [53]:
def roda_scraping_pagina_inicial():
    url = 'https://cursos.alura.com.br/forum/todos/1'
    soup = requisicao(url)
    nomes = soup.find_all('a', {'class': 'dashboard-category-name-text'})
    quantidades = soup.find_all('span', {'class': 'category-stats-item-number dashboard-stats-noReply-number'})
    
    dados = {}
    for indice in range(len(nomes)):
        dados[f'{nomes[indice].text.strip()}'] = f'{quantidades[indice].text.strip()}'
    qtd_topicos_off_topic = roda_scraping_off_topic()
    dados['Off-Topic'] = qtd_topicos_off_topic
    df = pd.DataFrame(dados.items(), columns = ['Categoria', 'Quantidade sem resposta'])
    return df

In [34]:
def roda_scraping_off_topic():
    url = 'https://cursos.alura.com.br/forum/categoria-offtopic/sem-resposta/'
    soup = requisicao(url)
    ultima_pagina = soup.find_all('a', {'class': 'paginationLink'})[-1].text
    url = url + ultima_pagina
    soup = requisicao(url)
    qtd_topicos = (int(ultima_pagina)-1)*20 + len(soup.find_all('a', {'class': 'forumList-item-subject-info-title-link'}))
    return qtd_topicos

In [54]:
roda_scraping_pagina_inicial()

,Categoria,Quantidade sem resposta
0,Mobile,34
1,Programação,2046
2,Front-end,128
3,DevOps,747
4,UX & Design,292
5,Data Science,444
6,Inovação & Gestão,718
7,Off-Topic,731
